In [7]:
import pandas as pd
import dotenv
dotenv.load_dotenv()
import os
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
import time

In [8]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or 'OPENAI_API_KEY'
PINECONE_API_KEY = os.getenv('PINECONE_SERVERLESS_API_KEY') or 'PINECONE_SERVERLESS_API_KEY'
index_name = 'langchain-retrieval-transcript'

namespace = 'langchain'

In [9]:
pinecone = Pinecone(api_key=PINECONE_API_KEY)
print(pinecone.list_indexes())

index = pinecone.Index(index_name)
index.describe_index_stats()

{'indexes': [{'dimension': 1536,
              'host': 'langchain-retrieval-transcript-kp69ciw.svc.apw5-4e34-81fa.pinecone.io',
              'metric': 'cosine',
              'name': 'langchain-retrieval-transcript',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'langchain': {'vector_count': 144}},
 'total_vector_count': 144}

In [10]:
#%pip install -U langchain-openai

In [16]:
from langchain.vectorstores import Pinecone
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI

# embedding model
model_name = 'text-embedding-ada-002'
embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

# pinecone store
text_field = 'text' # the field in the metadata that contains the text and would be used for retrieval
vector_store = Pinecone(
    index, embed, text_field, namespace=namespace)

# chat model
llm = ChatOpenAI(
	temperature=0,
	openai_api_key=OPENAI_API_KEY,
	model_name="gpt-3.5-turbo"
)

In [ ]:
# Using BasePromptTemplate  

from langchain_core.prompts.base import BasePromptTemplate

# define the prompt template
prompt_template = BasePromptTemplate(
    system_prompt="System: {system_message}\nHuman: {human_message}\n",
    human_prompt="System: {system_message}\nHuman: {human_message}\n",
    ai_prompt="System: {system_message}\nHuman: {human_message}\nAI: {ai_message}\n",
)

# define the schema
schema = {
    "system_message": SystemMessage,
    "human_message": HumanMessage,
    "ai_message": AIMessage,
}

# define the context
context = {
    "system_message": "Hello, I am a bot.",
    "human_message": "Hello, I am a human.",
    "ai_message": "Hello, I am an AI.",
}

# adding context to the prompt template
prompt_template.add_context(context)





In [ ]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain import PromptTemplate


template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Current conversation: {history}

Context: {context}	

Question: {input}
Answer: """

prompt_template = PromptTemplate(
    input_variables=['history','context','input'],
    template=template
)

# conversation chain
conversation = ConversationChain(
	llm=llm,
	prompt = prompt_template,
	memory=ConversationBufferWindowMemory(k=10),
	
)

In [19]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:
